# Entendimiento de Datos
En este cuaderno revisaremos dos aspectos grandes del entendimiento de datos: el perfilamiento y la calidad de los datos. Adicionalmente, se dan una serie de funciones para facilitar la manipulación de los datos.
* Inicio
    * Carga
    * Manipulación Básica

* Perfilamiento de Datos
    * Exploración
    * Visualización

* Calidad de Datos:
  * Completitud
  * Duplicados
  * Estandarización

* Resumenes automáticos para el entendimiento

Para la limpieza de los datos utilizaremos la libreria de **Pandas** (https://pandas.pydata.org/) y para la visualización de los datos, usaremos: **Seaborn**(https://seaborn.pydata.org/) y **Matplotlib** (https://matplotlib.org/).

## Los Datos
Trabajaremos con una base de datos de accidentes de BiciAlpes.

La base de datos original, la pueden encontrar aquí: **

# 1. Inicio

## 1.1 Carga

### 1.1.1 *Limpieza y lemantización*

In [ ]:
# Uninstall numba, pandas_profiling and visions to clear any existing installation
!pip uninstall numba -y
!pip uninstall pandas_profiling -y
!pip uninstall visions -y

# Install the necessary modules
!pip install numba==0.58.1
!pip install ydata-profiling
# Librerias generales
# Pandas
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Número máximo de filas a mostar
# Ranom seed
import numpy as np
np.random.seed(3301)

# Seaborn
import seaborn as sns

# Matplolib
%matplotlib inline
import matplotlib.pyplot as plt

# Plotly
!pip install plotly
import plotly.express as px

#Limpieza de datos

!pip install spacy
!python -m spacy download es_core_news_sm
%pip install nbformat

!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import re
import unicodedata

# Descargar e inicializar spaCy en español
import os
if not os.path.exists(spacy.util.get_package_path("es_core_news_sm")):
    spacy.cli.download("es_core_news_sm")

nlp = spacy.load("es_core_news_sm", disable=["ner", "textcat"])

In [ ]:
# Ubicación de la base de datos
db_location = 'fake_news_spanish.csv'

In [ ]:
# Cargar los datos
df = pd.read_csv(db_location, sep=';', encoding = "utf-8")

In [ ]:
# Dimensiones de los datos
df.shape

In [ ]:
# Ver los datos
display(df.sample(5)) # Muestra
#display(df_bicis.head(5)) # Primeras Filas
#display(df_bicis.tail(5)) # Ultimas Filas

Eliminación de la columna ID porque todos las filas lo tenián con el valor 'ID'

In [ ]:
df1 = df.drop("ID", axis = 1)

Revisamos cuantos valores nulos hay por columna

In [ ]:
valores_nulos = df1.isnull().sum()
print(valores_nulos)

Como en titulo hay 16 valores nulos, que es una cantida mínima comparado con la cantidad de elementos en el dataset. Los eliminamos

In [ ]:
df2 = df1.dropna()

Vamos ahora a revisar si hay elementos duplicados en la columna titulo

In [ ]:
duplicados_titulo = df2['Titulo'].duplicated().sum()
print(f"Duplicados según titulo: {duplicados_titulo}")

In [ ]:
valores_duplicados = df2[df2.duplicated(keep=False)]
valores_duplicados_ordenados = valores_duplicados.sort_values(by=df2.columns.tolist())
print(valores_duplicados_ordenados)

Después de analizar las filas repetidas y corroborar que el "label" es el mismo y que no se trataba de la misma noticia pero con una descripción diferente o algún diferenciador. Procedemos a eliminarlas

In [ ]:
df3 = df2.drop_duplicates(subset = ['Titulo'])

In [ ]:
duplicados_titulo = df3['Titulo'].duplicated().sum()
print(f"Duplicados según titulo: {duplicados_titulo}")

Ahora vamos procesar y limpíar el texto, para poder tener una mejor deteción de patrones

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

def limpiar_texto(texto):
    texto = texto.lower()
    tokens = word_tokenize(texto, language='spanish')

    stop_words = set(stopwords.words('spanish'))
    palabras_filtradas = [palabra for palabra in tokens if palabra not in stop_words]

    texto_limpio = ' '.join(palabras_filtradas)
    return texto_limpio


df3['Titulo'] = df3["Titulo"].apply(limpiar_texto)
df3['Descripcion'] = df3["Descripcion"].apply(limpiar_texto)

display(df3.head(5)) # Primeras Filas


In [ ]:
def lematizar_sin_tildes(texts):
    docs = nlp.pipe(texts, batch_size=500)
    textos_lematizados = [" ".join([token.lemma_ for token in doc if not token.is_punct])
                         for doc in docs]
    return [unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
            for texto in textos_lematizados]

df4 = df3.copy()
df4['Titulo'] = lematizar_sin_tildes(df4['Titulo'])
df4['Descripcion'] = lematizar_sin_tildes(df4['Descripcion'])

In [ ]:
display(df4.head(5)) # Primeras Filas

## 1.2 Manipulación Básica

En esta parte del Cuaderno la idea es que se familarice con algunos comandos que van a permitir manipular mejor los datos y avanzar en la comprensión de los mismos, muy de la mano del diccionario de datos.

In [ ]:
df4.dtypes

In [ ]:
# Resumen de las principales estadísticas de las variables numéricas
df4['Label'].describe()

#### 1.2.1 Datos de Fechas

In [ ]:
# la columna Fecha deberia ser fecha pero es object
df4.Fecha.tail(10)

In [ ]:
df5 = df4.copy()

df5['Fecha'] = pd.to_datetime(df5.Fecha, dayfirst= True, errors = 'coerce')
df5['Fecha'].tail(10)

In [ ]:
print(df5["Fecha"].isna().sum())

# 2. Vectorización del texto

In [ ]:
# Verificar si hay NaN en columnas de texto despues de la manipulacion de fechas
print("NaN en Titulo:", df5["Titulo"].isna().sum())
print("NaN en Descripcion:", df5["Descripcion"].isna().sum())

# Eliminar las filas donde hay NaN en las columnas de texto
df5 = df5.dropna(subset=["Titulo", "Descripcion"])

# Verificar que no haya mas NaN
print("NaN en Titulo despues de eliminar:", df5["Titulo"].isna().sum())
print("NaN en Descripcion despues de eliminar:", df5["Descripcion"].isna().sum())


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b[a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]{2,}\b", max_features=5000)
X_tfidf = vectorizer.fit_transform(df5["Titulo"] + " " + df5["Descripcion"])

# Convertir a DataFrame
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
print(tfidf_df.head())

# 3. Visualización de los datos

### 2.4.1 Diagramas de Temporales

Distribución de las noticias según si son verdaderas o falsas

In [ ]:
conteo_clases = df5["Label"].value_counts()

mapeo_etiquetas = {0: "Noticias Falsas", 1: "Noticias Verdaderas"}
fig = px.pie(names=conteo_clases.index.map(mapeo_etiquetas), values=conteo_clases.values,
             title=f'Distribución de noticias falsas y verdaderas ({df.shape[0]} noticias)')

fig

Dado que la distribución entre noticias falsas y verdaderas es relativamente equilibrada, podemos trabajar con los datos sin necesidad de ajustar el balance de clases.

---
Analisis para identificar si la fecha esta directamente relacionada con la veracidad de una noticia

In [ ]:
print("Rango de fechas:", df5["Fecha"].min(), "a", df5["Fecha"].max())


# Distribución de noticias falsas vs. verdaderas por año
plt.figure(figsize=(10,5))
df5.groupby([df5["Fecha"].dt.year, "Label"]).size().unstack().rename(columns={0: "Falsas", 1: "Verdaderas"}).plot(kind="bar", stacked=True, figsize=(10,5))
plt.title("Distribución de noticias falsas y verdaderas por año")
plt.xlabel("Año")
plt.ylabel("Cantidad de noticias")
plt.legend(["Verdaderas (1)", "Falsas (0)"])
plt.show()

# Análisis de correlación entre año y etiqueta de noticia
correlacion = df5["Fecha"].dt.year.corr(df["Label"])
print("Correlación entre año y etiqueta de noticia:", correlacion)

## 4. Reportes Automáticos para realizar el entendimiento de los datos

Para los reportes automáticos, se puede usar al herramienta de pandas profiling.


Para cada columna, genera las siguientes estadísticas, si son relevantes para el tipo de columna, se presentan en un informe HTML interactivo:

1. Inferencia de tipo: detecta los tipos de columnas en un dataframe.
2. Esenciales: tipo, valores únicos, valores faltantes.
3. Estadísticas de cuantiles como valor mínimo, Q1, mediana, Q3, máximo, rango, rango intercuartílico. Esta opción es bastante útil para identificar datos atípicos.
4. Estadísticas descriptivas como media, moda, desviación estándar, suma, desviación absoluta mediana, coeficiente de variación, curtosis, asimetría.
5. Valores más frecuentes.
6. Histogramas.
7. Correlaciones destacando variables altamente correlacionadas, matrices de Spearman, Pearson y Kendall. Esto permite descubrir relaciones entre atributos.
8. Matriz de valores faltantes, recuento, mapa de calor y dendrograma de valores faltantes

Tomado de la librería oficial de pandas_profiling en [github](https://github.com/pandas-profiling/pandas-profiling)

Lo más importante al utilizar esta librería es recordar que lo fundamental son los análisis que hagamos sobre estos reportes.

In [ ]:
import pandas_profiling

profiling =pandas_profiling.ProfileReport(df5)
profiling

In [ ]:
profiling.to_file("Proyecto1_db_profile.html")

# Modelos

## Modelo basado en reglas (RIPPER)

Para clasificar noticias falsas utilizando un modelo basado en reglas, se escogio el algoritmo RIPPER (Repeated Incremental Pruning to Produce Error Reduction), ya que es una opción eficiente y escalable para conjuntos de datos grandes, como el de 60,000 registros. 

Algunas de sus principales ventajas son:  

- **Simplicidad y explicabilidad**: Genera reglas fácilmente interpretables, lo que permite comprender mejor los patrones asociados a las noticias falsas.  
- **Eficiencia en grandes volúmenes de datos**: Está diseñado para procesar grandes cantidades de información sin riesgo de sobreajuste.  
- **Manejo de datos desbalanceados**: Puede ajustar la cobertura de las reglas para reducir el sesgo hacia la clase mayoritaria, mejorando así la precisión en la clasificación.

### Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

%pip install wittgenstein
import wittgenstein as lw

### Modelo

In [ ]:

pca = PCA(n_components=100, random_state=42)
X_reduced = pca.fit_transform(tfidf_df)

X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, df5['Label'], test_size=0.2, random_state=42)

X_train_sample = X_train
y_train_sample = y_train

print(y_train_sample.value_counts())
pos_class = 1 if y_train_sample.value_counts().idxmax() == 1 else 0

# Entrenar modelo RIPPER
ripper_classifier = lw.RIPPER()
ripper_classifier.fit(X_train_sample, y_train_sample, pos_class=pos_class)

# Hacer predicciones
y_pred = ripper_classifier.predict(X_test)

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# Evaluar modelo
print(classification_report(y_test, y_pred))